<a href="https://colab.research.google.com/github/janaahmeed/codealpha_tasks/blob/main/code_Alpha_task2_(Emotion_Recognition_from_Speech_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import sklearn
import joblib

After framing, each frame is multiplied by a Hamming window to reduce spectral leakage before silence detection and further feature extraction.  
# When you use librosa.feature.mfcc, framing + windowing (Hamming) are already applied internally by default.

y>> the audio time-series signal , Compute MFCC features from audio signal lf sampled at fs Hz
y(t) = audio_signal
mfcc(y=audio_signal)

Always join files with root, never dirs.

In [16]:

import random
import librosa
import numpy as np
def apply_random_augmentation(signal, sr):
    augmentations = ["none", "stretch", "shift", "pitch", "noise"]
    choice = random.choice(augmentations)

    noise_factor = 0.005

    if choice == "noise":
        noise_factor = 0.005
        signal = signal + np.random.randn(len(signal)) * noise_factor

    elif choice == "stretch":
        rate = np.random.uniform(0.8, 1.2)
        signal = librosa.effects.time_stretch(signal, rate=rate)

    elif choice == "shift":
        shift = int(np.random.uniform(-0.2, 0.2) * len(signal))
        signal = np.roll(signal, shift)

    elif choice == "pitch":
        signal = librosa.effects.pitch_shift(
            signal,
            sr=sr,
            n_steps=np.random.uniform(-2, 2)
        )

    return signal



def load_audio(path, sr=44100):
    signal, sr = librosa.load(path, sr=sr, offset=0.5, duration=2.5)

    signal = apply_random_augmentation(signal, sr)

    return signal, sr


In [19]:
#
import os
import kagglehub

DATA_FILE = "ravdess-emotional-speech-audio"

if os.path.exists(DATA_FILE):
    print("Loading cached features...")

    x, y ,speakers = joblib.load(DATA_FILE)

else:
    print("Processing images (one-time)...")

# Download latest version
    path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")


    print("Path to dataset files:", path)
    x,y, speakers=[],[],[]





for root, dirs, files in os.walk(path):
    for _file in files:
        if _file.endswith(".wav"):
            file_path = os.path.join(root, _file)
            aug_data ,sr  =load_audio(file_path )

            y_trimmed, _ = librosa.effects.trim(aug_data ,top_db=25)  # 20–30 is common for speech

            mel_spec = librosa.feature.melspectrogram(
            y=y_trimmed,
            sr=sr,
            n_fft=2048,
            hop_length=512,
            n_mels=128
                    )

        # 3. Convert power to decibels (log scale)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

        # 4. Transpose if you are using LSTMs (Time steps first)
        # Resulting shape will be (251, 128) based on your target
        mel_spec_db = mel_spec_db.T

        x.append(mel_spec_db)

        # Extract label (assuming RAVDESS-style naming: 03-01-01-...)
        emo_str = _file.split("-")[2]
        y.append(int(emo_str))




joblib.dump((x, y ,speakers), DATA_FILE)
print("Features saved.")
#(n_mfcc, time_frames) >>shape mfcc
# l your mel_spec_db arrays have the same first dimension (128) But different time steps (216, 210, 198, etc.)
#solution to avoid : (n_samples, 128, ?)
 #This forces NumPy to:create a 1D object array first ,Then insert each 2D MFCC/mel matrix as a separate object ,No broadcasting attempt happens


max_len = 251
padded_x = []

for sample in x:
    # sample shape: (time, 128)

    if sample.shape[0] < max_len:
        pad_width = max_len - sample.shape[0]
        sample = np.pad(
            sample,
            pad_width=((0, pad_width), (0, 0)),
            mode='constant'
        )
    else:
        sample = sample[:max_len, :]

    padded_x.append(sample)

# NOW convert safely
x = np.array(padded_x, dtype='float32')
x=x[..., np.newaxis]

y = np.array(y)

print(x.shape, y.shape)




Loading cached features...
Features saved.
(8640, 251, 128, 1) (8640,)


In [20]:
print("X shape:", x.shape)
print("y shape:", y.shape)
print("Unique labels:", np.unique(y))
print("Random baseline:", 1 / len(np.unique(y)))


X shape: (8640, 251, 128, 1)
y shape: (8640,)
Unique labels: [1 2 3 4 5 6 7 8]
Random baseline: 0.125


In [ ]:
import seaborn as sns
sns.countplot(np.unique(y))
plt.show()

In [21]:
x.shape

(8640, 251, 128, 1)

//CNN / LSTM

In [22]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_encoded = le.fit_transform(y)
#y_encoded = to_categorical(y_encoded ,num_classes=7)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x,y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)






MFCC features have limited frequency resolution, so aggressive pooling across both axes may collapse feature maps.
We therefore apply pooling mainly along the time axis and preserve frequency information.

model.add(Reshape((-1, x.shape[2] * 128)))
Why?

-1 lets Keras infer time steps automatically

Works even if you change pooling later

model.add(MaxPooling2D(pool_size=(1, 2)))
This:

reduces noise

keeps emotion dynamics

stabilizes LSTM learning

Frequency pooling:

smooths spectral noise

improves generalization

stabilizes training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# Conv → BN → ReLU → Pool.
model = Sequential()

# Input Layer: 128 Mel bands x 251 Time bins x 1 Grayscale channel
model.add(Input(shape=(128, 251, 1)))

# --- CNN Block 1 ---
# 32 filters, 5x5 kernel, stride (5,3)>> image
model.add(Conv2D(32, (5, 5), strides=(5, 3), padding='valid'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))

# --- CNN Block 2 ---
# 64 filters, 3x3 kernel, stride (3,2)
model.add(Conv2D(64, (3, 3), strides=(3, 2), padding='valid'))
model.add(BatchNormalization())
model.add(ReLU())

# --- CNN Block 3 ---
# 128 filters, 3x3 kernel, 'same' padding
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))

# --- Transition to Classifier ---
model.add(Flatten())

# --- Dense Layer 1 ---
model.add(Dense(256))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.3))

# --- Dense Layer 2 ---
model.add(Dense(128))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.3))

# --- Output Layer ---

model.add(Dense(8, activation='softmax'))

# Compilation
opt = Adam(learning_rate=0.0001)
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Prepare for LSTM >> flattten inseadreshape (frq * channels )
#model.add(TimeDistributed(Flatten()))

# LSTM blocks
#model.add(Bidirectional(LSTM(128, return_sequences=True)))

#model.add(Dropout(0.3))

#model.add(Bidirectional(LSTM(256, return_sequences=False)))

#model.add(Dropout(0.3))

#model.add(GlobalAveragePooling1D())


In [39]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Flatten, Dense, Input
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Flatten, Dense, Input

inputs = Input(shape=(251, 128, 1))

x_layer = Conv2D(32, (5,5), strides=(5,3), padding='valid')(inputs)
x_layer = BatchNormalization()(x_layer)
x_layer = ReLU()(x_layer)
x_layer = MaxPooling2D((2,2))(x_layer)

x_layer = Conv2D(64, (3,3), strides=(3,2), padding='valid')(x_layer)
x_layer = BatchNormalization()(x_layer)
x_layer = ReLU()(x_layer)

x_layer = Conv2D(128, (3,3), padding='same')(x_layer)
x_layer = BatchNormalization()(x_layer)
x_layer = ReLU()(x_layer)
x_layer = MaxPooling2D((2,2))(x_layer)

flatten_output = Flatten(name='flatten_layer')(x_layer)

dense = Dense(256, activation='relu')(flatten_output)
dense = Dense(128, activation='relu')(dense)
outputs = Dense(8, activation='softmax')(dense)

#model = Model(inputs=inputs, outputs=outputs)

# Feature extractor
feature_extractor = Model(inputs=inputs, outputs=flatten_output)

# 2. Define the feature extractor
# We map the model's input to the output of the 'flatten_layer'
feature_extractor = Model(
    inputs=model.input,
    outputs=model.get_layer('flatten_layer').output
)
# 3. Predict to get the features (ensure x is shape (samples, 128, 251, 1))
X_features = feature_extractor.predict(x)

# 4. Feature Selection (Picking top 100 features)
selector = SelectKBest(score_func=mutual_info_classif, k=500)
X_selected = selector.fit_transform(X_features, y)

# 5. Scaling (Essential for SVM)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# 6. Initialize and Train SVM
svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)
svm_classifier.fit(X_scaled, y)

# 7. Results
accuracy = svm_classifier.score(X_scaled, y)
print(f"SVM Classification Accuracy: {accuracy * 100:.2f}%")

270/270 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
SVM Classification Accuracy: 71.72%


-(batch, 7) from LSTM
-matches output with sparse_categorical_crossentropy


In [30]:
print(y_test.dtype)

int64


In [31]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)

class_weights = dict(enumerate(class_weights))
print(class_weights)